# Graph9: Agent4
## Task
    - Create an AI Agentic System that can speed up drafting of documents, emails etc. 
    - Integrate Human-Ai integration. 
    - Feedback should be provided by a human and feeback implemented by the Ai until satisfaction by human

In [5]:
from typing import TypedDict,  Annotated, Sequence
from langchain_core.messages import BaseMessage, ToolMessage, SystemMessage, HumanMessage
from langchain_core.tools import tool
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from openai import OpenAI
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
load_dotenv()

# Open router server/api
api_key = os.getenv("open_router_api_key")
model_name="meta-llama/llama-4-maverick:free"
model_name2 = "google/gemini-2.0-flash-001"
base_url="https://openrouter.ai/api/v1"

llm = ChatOpenAI(
  openai_api_key=api_key,
  openai_api_base=base_url,
  model_name=model_name2,
)

# Google ai
llm2 = ChatGoogleGenerativeAI(model="gemini-2.0-flash")



C:\Users\maxdb\AppData\Local\Temp\ipykernel_2100\1766644248.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
# Using a global variable to Grant tool access to state
document_content = ""

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [3]:
@tool
def update(content: str) -> str:
    """Updates the document with the provided content."""
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is:\n{document_content}"

@tool
def save(filename: str) -> str:
    """Save the current document to a text file and finish the process.

    Args:
        filename: Name for the text file.
    """
    global document_content

    if not filename.endswith(".txt"):
        filename = f"{filename}.txt"
    
    try:
        with open(filename, "w") as file:
            file.write(document_content)
        print(f"\nDocument has been saved to: {filename}")
        return f"Document has been saved successfully to '{filename}'."
    except Exception as e:
        return f"Error saving document: {str(e)}"
    
tools = [update, save]
model = llm2.bind_tools(tools)

In [ ]:
def our_agent(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.
                                    
    - If the user wants to update or modify content, use the "update" tool with the complete updated content.
    - If the user wants to save and finish, you need to use the "save" tool.
    - Make sure to always show the current document state after modifications.

    The current document content is: {document_content}                                  
    """)

    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)
    else:
        user_input = input("\nWhat would you like to do with the document? ")
        print(f"\n🤵 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_prompt] + list(state["messages"]) + [user_message]
    response = model.invoke(all_messages)
    print(f"\n🤖 AI: {response.content}")

    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"🛠️ USING TOOLS: {[tc["name"] for tc in response.tool_calls]}")

    return {"messages": list(state["messages"] + [user_message, response])}

def should_loop(state: AgentState) -> str:
    """Determine if we should continue or end the conversation"""

    messages = state["messages"]

    if not messages:
        return "continue"
    
    for message in reversed(messages):
        if(isinstance(message, ToolMessage) and "saved" in message.content.lower() and "document" in message.content.lower()):
            return "end"